In [1]:
import networkx as nx
import Graph
import Classifier
from gensim.models import Word2Vec


In [2]:
#Open and parse dataset
G, subs_coo, subs = Graph.parse_mat_file('blogcatalog.mat')

In [3]:
#Build corpus
corpus = Graph.build_walk_corpus(G=G, max_paths=80, path_len=40)

In [4]:
#Train model
#ToDO: try negative sampling (hs=0)
model = Word2Vec(size = 128, window=10, sg=1, min_count=0, hs=1, compute_loss=True)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.iter)

/Users/ass6ash/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


(40357636, 41248000)

In [5]:
#Save w2v embeddings
word_vec = model.wv
word_vec.save_word2vec_format(binary=False,fname='word2vec.txt')

In [3]:
#Sometimes the model doesn't predict anything at all for some inputs. Its either the model's fault or that user has no subscriptions at
#all, in that case the model is predicting properly but of course a zero output would raise exceptions during sklearn's
#F1 score function.
#Currently evaluating performance with OVR Logistic Regression.
results = Classifier.evaluate(G, subs_coo)

/Users/ass6ash/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/ass6ash/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/ass6ash/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [4]:
trainsize = [0.2,0.5,0.8, 0.9]
for (name,res) in results.items():
    print("Classifier : ",name)
    for (tr_size,res) in zip(trainsize,res):
        print("\tTraining size : ",tr_size)
        print("\t\tMicro F1: ",res[0])
        print("\t\tMacro F1: ",res[1])

Classifier :  Logistic_classifier
	Training size :  0.2
		Micro F1:  0.29341627786417834
		Macro F1:  0.21410135288977103
	Training size :  0.5
		Micro F1:  0.29826749826749827
		Macro F1:  0.2343146163356684
	Training size :  0.8
		Micro F1:  0.31282768262845156
		Macro F1:  0.2539729821864884
	Training size :  0.9
		Micro F1:  0.3054571226080794
		Macro F1:  0.2393629968703233


In [ ]:
#Results don't quite match the ones in the paper. Might have to tune the parameters to get optimal performance.